In [1]:
import sys
sys.path.insert(0, r"C:\Users\PC\Desktop\graphglue - Copie - Copie")
from graphglue.core.graph import Graph


In [3]:
# ---------- Setup ----------
G = Graph(directed=True)
conditions = ["Healthy", "Stressed", "Disease"]
for c in conditions:
    G.add_layer(c, condition=c)

# Entities
proteins = [f"P{i}" for i in range(1, 151)]   # P1..P150
transcripts = [f"T{i}" for i in range(1, 61)]  # T1..T60 (treat as vertices)
enz_edge_entities = [f"edge_rxn_{i}" for i in range(1, 11)]  # edge-entities for reactions

# Seed some vertex attributes
for p in proteins[:10]:
    G.add_vertex(p, layer="Healthy", family="kinase")
for p in proteins[10:]:
    G.add_vertex(p, layer="Healthy")
for t in transcripts:
    G.add_vertex(t, layer="Healthy", kind="transcript")
for ee in enz_edge_entities:
    G.add_edge_entity(ee, layer="Healthy", role="enzyme")

# Propagate initial vertices to all layers (cheaply)
for lid in ["Stressed", "Disease"]:
    G._layers[lid]["vertices"].update(G._layers["Healthy"]["vertices"])

In [5]:
# ---------- Build PPI edges in all layers ----------
import random

def rand_weight(base=1.0, jitter=0.5):
    return max(0.05, base + (random.random() - 0.5) * 2 * jitter)

ppis = []
for _ in range(320):
    u, v = random.sample(proteins, 2)
    w = rand_weight(1.2, 0.6)
    e = G.add_edge(u, v, layer="Healthy", weight=w, edge_directed=False)
    ppis.append(e)

# Stress/disease layer variants (override per-layer weights)
for eid in ppis:
    # Stressed: mostly +10% with jitter
    G.add_edge_to_layer("Stressed", eid)
    G.set_edge_layer_attrs("Stressed", eid, weight=G.edge_weights[eid] * rand_weight(1.10, 0.1))
    # Disease: some edges get weaker; others stronger
    G.add_edge_to_layer("Disease", eid)
    factor = 0.7 if random.random() < 0.4 else 1.3
    G.set_edge_layer_attrs("Disease", eid, weight=G.edge_weights[eid] * rand_weight(factor, 0.15))

In [7]:
# ---------- Complexes as undirected hyperedges ----------
complexes = []
for _ in range(12):
    members = set(random.sample(proteins, random.choice([3, 4, 5])))
    hid = G.add_hyperedge(members=members, layer="Healthy", weight=rand_weight(1.0, 0.2))
    complexes.append(hid)
    # complex exists in all layers (same membership)
    for lid in ["Stressed", "Disease"]:
        G.add_edge_to_layer(lid, hid)

In [9]:
# ---------- Directed signaling cascades as hyperedges ----------
cascades = []
while len(cascades) < 8:
    head = set(random.sample(proteins, random.choice([1, 2])))
    tail = set(random.sample(proteins, random.choice([2, 3, 4])))
    if head & tail:
        continue  # resample until disjoint
    hid = G.add_hyperedge(head=head, tail=tail, layer="Healthy", weight=rand_weight(1.0, 0.4))
    cascades.append(hid)
    for lid in ["Stressed", "Disease"]:
        G.add_edge_to_layer(lid, hid)

In [11]:
# ---------- Reactions connecting vertices to edge-entities ----------
for ee in enz_edge_entities:
    s, t = random.sample(proteins, 2)
    G.add_edge(s, ee, layer="Healthy", edge_type="vertex_edge", weight=1.0 + random.random())
    G.add_edge(ee, t, layer="Healthy", edge_type="vertex_edge", weight=1.0 + random.random())
    # propagate across layers
    for lid in ["Stressed", "Disease"]:
        G._layers[lid]["edges"].update(G._layers["Healthy"]["edges"])

In [13]:
# ---------- Basic sanity ----------
print("vertices:", G.number_of_vertices(), "Edges:", G.number_of_edges())

# Only true "vertices" are counted by number_of_vertices() (proteins + transcripts)
expected_vertices = len(set(proteins)) + len(set(transcripts))  # 150 + 60 = 210
assert G.number_of_vertices() >= expected_vertices, f"Expected ≥{expected_vertices}, got {G.number_of_vertices()}"

# Edge-entities are tracked as entity_type == 'edge' (not included in number_of_vertices)
edge_entity_ids = set(enz_edge_entities)
edge_entity_count = sum(1 for _id, et in G.entity_types.items() if et == "edge" and _id in edge_entity_ids)
assert edge_entity_count == len(edge_entity_ids), f"Expected {len(edge_entity_ids)} edge-entities, got {edge_entity_count}"

# Edges: PPIs (320) + complexes (12) + cascades (8) + reaction links (10*2) = 360 minimum
assert G.number_of_edges() >= 320 + 12 + 8 + (10 * 2)


vertices: 210 Edges: 360


In [15]:
# ---------- Views & top edges by condition ----------
import polars as pl

for cond in conditions:
    EV = G.edges_view(layer=cond, resolved_weight=True)
    print(f"[{cond}] edges_view rows =", EV.height)
    top = (
        EV
        .filter(pl.col("kind") == "binary")
        .sort("effective_weight", descending=True)
        .select(["edge_id", "source", "target", "effective_weight"])
        .head(5)
    )
    print(f"\nTop 5 binary edges by effective_weight in {cond}:")
    print(top)

[Healthy] edges_view rows = 360

Top 5 binary edges by effective_weight in Healthy:
shape: (5, 4)
┌──────────┬────────────┬────────────┬──────────────────┐
│ edge_id  ┆ source     ┆ target     ┆ effective_weight │
│ ---      ┆ ---        ┆ ---        ┆ ---              │
│ str      ┆ str        ┆ str        ┆ f64              │
╞══════════╪════════════╪════════════╪══════════════════╡
│ edge_356 ┆ P19        ┆ edge_rxn_9 ┆ 1.952197         │
│ edge_354 ┆ P92        ┆ edge_rxn_8 ┆ 1.943664         │
│ edge_348 ┆ P143       ┆ edge_rxn_5 ┆ 1.936892         │
│ edge_342 ┆ P8         ┆ edge_rxn_2 ┆ 1.906609         │
│ edge_347 ┆ edge_rxn_4 ┆ P105       ┆ 1.867855         │
└──────────┴────────────┴────────────┴──────────────────┘
[Stressed] edges_view rows = 360

Top 5 binary edges by effective_weight in Stressed:
shape: (5, 4)
┌──────────┬────────────┬────────────┬──────────────────┐
│ edge_id  ┆ source     ┆ target     ┆ effective_weight │
│ ---      ┆ ---        ┆ ---        ┆ ---      

In [17]:
# ---------- Layer analytics ----------
stats = G.layer_statistics()
print("\nLayer stats:", stats)

conserved = G.conserved_edges(min_layers=3)  # present in all 3 conditions
print("\nConserved edges (in all conditions):", len(conserved))

disease_specific = G.layer_specific_edges("Disease")
print("Disease-specific edges:", len(disease_specific))

changes = G.temporal_dynamics(["Healthy", "Stressed", "Disease"], metric="edge_change")
print("\nTemporal edge changes (Healthy→Stressed→Disease):", changes)
assert len(changes) == 2


Layer stats: {'Healthy': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Healthy'}}, 'Stressed': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Stressed'}}, 'Disease': {'vertices': 220, 'edges': 360, 'attributes': {'condition': 'Disease'}}}

Conserved edges (in all conditions): 360
Disease-specific edges: 0

Temporal edge changes (Healthy→Stressed→Disease): [{'added': 0, 'removed': 0, 'net_change': 0}, {'added': 0, 'removed': 0, 'net_change': 0}]


In [19]:
# ---------- Presence queries ----------
some_e = next(iter(G.edge_to_idx.keys()))
print("\nEdge presence for", some_e, ":", G.edge_presence_across_layers(edge_id=some_e))
some_p = random.choice(proteins)
print("vertex presence for", some_p, ":", G.vertex_presence_across_layers(some_p))


Edge presence for edge_0 : ['Healthy', 'Stressed', 'Disease']
vertex presence for P101 : ['Healthy', 'Stressed', 'Disease']


In [21]:
# ---------- Traversal checks ----------
q = random.choice(proteins)
print(f"\nNeighbors({q}) =>", G.neighbors(q)[:10])
print(f"Out({q}) =>", G.out_neighbors(q)[:10])
print(f"In({q}) =>", G.in_neighbors(q)[:10])


Neighbors(P141) => ['P48', 'P56', 'P53', 'P149', 'P139', 'P64', 'P82', 'P143', 'P1', 'P127']
Out(P141) => ['P48', 'P56', 'P53', 'P149', 'P139', 'P64', 'P1', 'P127']
In(P141) => ['P48', 'P56', 'P53', 'P149', 'P139', 'P64', 'P82', 'P143', 'P1', 'P127']


In [23]:
# ---------- Subgraph slice & copy ----------
H = G.subgraph_from_layer("Disease", resolve_layer_weights=True)
assert set(H.vertices()).issubset(set(G.vertices()))
assert set(H.edges()).issubset(set(G.edges()))
print("\nDisease subgraph: vertices =", H.number_of_vertices(), "edges =", H.number_of_edges())

K = G.copy()
assert set(K.vertices()) == set(G.vertices())
assert set(K.edges()) == set(G.edges())
# hyperedge shape preserved
any_hyper = next(e for e,k in G.edge_kind.items() if k == "hyper")
assert K.edge_kind.get(any_hyper) == "hyper"
# layer sets preserved
for lid in G.list_layers(include_default=True):
    assert K._layers[lid]["vertices"] == G._layers[lid]["vertices"]
    assert K._layers[lid]["edges"] == G._layers[lid]["edges"]
print("copy() OK")


Disease subgraph: vertices = 210 edges = 360
copy() OK


In [24]:
# ---------- Remove operations stress ----------
to_drop_vertices = random.sample(proteins, 5)
for n in to_drop_vertices:
    if n in G.entity_to_idx:
        G.remove_vertex(n)
print("\nAfter removing 5 proteins: vertices =", G.number_of_vertices(), "edges =", G.number_of_edges())

to_drop_edges = list(G.edge_to_idx.keys())[:10]
for eid in to_drop_edges:
    if eid in G.edge_to_idx:
        G.remove_edge(eid)
print("After removing 10 edges: vertices =", G.number_of_vertices(), "edges =", G.number_of_edges())


After removing 5 proteins: vertices = 205 edges = 335
After removing 10 edges: vertices = 205 edges = 325


In [27]:
# ---------- Audit & memory ----------
audit = G.audit_attributes()
print("\nAudit:", audit)
mem_bytes = G.memory_usage()
print("Approx memory usage (bytes):", int(mem_bytes))
assert mem_bytes > 0

print("\nReality-check finished ✅")


Audit: {'extra_vertex_rows': ['edge_rxn_7', 'edge_rxn_5', 'edge_rxn_1', 'edge_rxn_8', 'edge_rxn_6', 'edge_rxn_3', 'edge_rxn_4', 'edge_rxn_10', 'edge_rxn_2', 'edge_rxn_9'], 'extra_edge_rows': [], 'missing_vertex_rows': [], 'missing_edge_rows': ['edge_249', 'edge_345', 'edge_16', 'edge_278', 'edge_236', 'edge_169', 'edge_107', 'edge_21', 'edge_304', 'edge_232', 'edge_352', 'edge_18', 'edge_106', 'edge_153', 'edge_55', 'edge_85', 'edge_243', 'edge_276', 'edge_295', 'edge_357', 'edge_161', 'edge_290', 'edge_359', 'edge_289', 'edge_256', 'edge_59', 'edge_288', 'edge_239', 'edge_32', 'edge_268', 'edge_132', 'edge_318', 'edge_212', 'edge_358', 'edge_137', 'edge_326', 'edge_130', 'edge_353', 'edge_22', 'edge_71', 'edge_124', 'edge_47', 'edge_317', 'edge_155', 'edge_96', 'edge_252', 'edge_248', 'edge_41', 'edge_303', 'edge_125', 'edge_313', 'edge_351', 'edge_95', 'edge_211', 'edge_216', 'edge_115', 'edge_156', 'edge_13', 'edge_37', 'edge_168', 'edge_103', 'edge_140', 'edge_233', 'edge_301', 'e

In [29]:
events = G.history()           # list[dict]
df = G.history(as_df=True)     # Polars DF [DataFrame]

In [31]:
print(df.head())
events[:3]

shape: (5, 10)
┌─────────┬─────────────────────────────┬─────────┬─────────────────┬───┬────────┬───────────┬─────────┬────────────┐
│ version ┆ ts_utc                      ┆ mono_ns ┆ op              ┆ … ┆ result ┆ vertex_id ┆ layer   ┆ attributes │
│ ---     ┆ ---                         ┆ ---     ┆ ---             ┆   ┆ ---    ┆ ---       ┆ ---     ┆ ---        │
│ i64     ┆ str                         ┆ i64     ┆ str             ┆   ┆ str    ┆ str       ┆ str     ┆ struct[1]  │
╞═════════╪═════════════════════════════╪═════════╪═════════════════╪═══╪════════╪═══════════╪═════════╪════════════╡
│ 1       ┆ 2025-10-04T21:12:50.341772Z ┆ 2803400 ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 2       ┆ 2025-10-04T21:12:50.342772Z ┆ 3174500 ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 3       ┆ 2025-10-04T21:12:50.342772Z ┆ 3560300 ┆ set_layer_attrs ┆ … ┆ null   ┆ null      ┆ null    ┆ null       │
│ 4       ┆ 2025-10-04T21:12:50.343773Z ┆

[{'version': 1,
  'ts_utc': '2025-10-04T21:12:50.341772Z',
  'mono_ns': 2803400,
  'op': 'set_layer_attrs',
  'layer_id': 'Healthy',
  'attrs': {'condition': 'Healthy'},
  'result': None},
 {'version': 2,
  'ts_utc': '2025-10-04T21:12:50.342772Z',
  'mono_ns': 3174500,
  'op': 'set_layer_attrs',
  'layer_id': 'Stressed',
  'attrs': {'condition': 'Stressed'},
  'result': None},
 {'version': 3,
  'ts_utc': '2025-10-04T21:12:50.342772Z',
  'mono_ns': 3560300,
  'op': 'set_layer_attrs',
  'layer_id': 'Disease',
  'attrs': {'condition': 'Disease'},
  'result': None}]

In [33]:
import sys, pathlib

# add repo root to Python path
sys.path.append(str(pathlib.Path.cwd().parent))
from graphglue.io import csv as graph_csv

In [35]:
csv1_path = "csv1_edges.csv"
pl.DataFrame({
    "source": ["A","A","B","C","D"],
    "target": ["B","C","C","D","A"],
    "weight": [1, 2, 3, 1, 5],
    "directed": [True, True, False, True, True],
    "layer": ["L1","L1","L1","L2","L2"],
}) #.write_csv(csv1_path)

#pl.read_csv(csv1_path).head()


source,target,weight,directed,layer
str,str,i64,bool,str
"""A""","""B""",1,true,"""L1"""
"""A""","""C""",2,true,"""L1"""
"""B""","""C""",3,false,"""L1"""
"""C""","""D""",1,true,"""L2"""
"""D""","""A""",5,true,"""L2"""


In [37]:
G = graph_csv.load_csv_to_graph(
    csv1_path,
    schema="auto",            # or 'edge_list'/'incidence'/'adjacency'/'hyperedge'/'lil'
    default_layer=None,       # fallback if no layer column is present
    default_directed=None,    # fallback if no directed column and cannot infer
    default_weight=1.0,
)

# Quick sanity: show first rows of an edges view (columns depend on your Graph implementation)
edges = G.edges_view(layer=None, include_directed=True, resolved_weight=True)
edges.head()


FileNotFoundError: Le fichier spécifié est introuvable. (os error 2): csv1_edges.csv

In [39]:
# Count entities and edges (attribute names based on your class; adjust if different)
num_entities = G.global_entity_count        # vertices + edge-entities
num_edges    = G.global_edge_count          # binary + hyper

print("entities:", num_entities, "edges:", num_edges)

# A light “degree” summary from edges_view for binary edges only (skip hyper)
df = G.edges_view(include_directed=True, resolved_weight=True)

# Try to find endpoint column names robustly
cols = {c.lower(): c for c in df.columns}
# Common possibilities:
src_col = next((cols[c] for c in ["source","src","u","from"]), None)
dst_col = next((cols[c] for c in ["target","dst","v","to"]), None)

if src_col and dst_col:
    # out-degree (directed) / degree (undirected)
    out_deg = df.group_by(src_col).len().rename({src_col: "vertex", "len": "out_degree"})
    in_deg  = df.group_by(dst_col).len().rename({dst_col: "vertex", "len": "in_degree"})
    deg = out_deg.join(in_deg, on="vertex", how="outer").fill_null(0)
    deg = deg.with_columns((pl.col("out_degree")+pl.col("in_degree")).alias("total_degree"))
    deg.sort("total_degree", descending=True).head(10)
else:
    print("Skip degree summary: endpoint columns not found in edges_view output (likely hyperedge-only or different schema).")


entities: <bound method Graph.global_entity_count of <graphglue.core.graph.Graph object at 0x00000203A16635C0>> edges: <bound method Graph.global_edge_count of <graphglue.core.graph.Graph object at 0x00000203A16635C0>>


C:\Users\pc\AppData\Local\Temp\ipykernel_2840\2468729116.py:20: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  deg = out_deg.join(in_deg, on="vertex", how="outer").fill_null(0)


In [41]:
# Add a new vertex and an edge on layer L3
G.add_vertex("E")
eid = G.add_edge("E", "A", layer="L3", directed=True, weight=2.5)

# Per-layer weight override example:
G.set_edge_layer_attrs("L3", eid, weight=3.0)

# Inspect the updated edges
G.edges_view(include_directed=True, resolved_weight=True).tail()


edge_id,kind,directed,global_weight,effective_weight,source,target,edge_type
str,str,bool,f64,f64,str,str,str
"""edge_356""","""binary""",true,1.952197,1.952197,"""P19""","""edge_rxn_9""","""vertex_edge"""
"""edge_357""","""binary""",true,1.279399,1.279399,"""edge_rxn_9""","""P130""","""vertex_edge"""
"""edge_358""","""binary""",true,1.437897,1.437897,"""P91""","""edge_rxn_10""","""vertex_edge"""
"""edge_359""","""binary""",true,1.376339,1.376339,"""edge_rxn_10""","""P94""","""vertex_edge"""
"""edge_360""","""binary""",true,2.5,2.5,"""E""","""A""","""regular"""


In [43]:
csv2_path = "csv2_edges_view.csv"
G.edges_view(layer=None, include_directed=True, resolved_weight=True)
csv2_path


'csv2_edges_view.csv'

In [45]:
def export_edge_list_csv(G, path, layer=None):
    df = G.edges_view(layer=layer, include_directed=True, resolved_weight=True)
    cols = {c.lower(): c for c in df.columns}
    src_col = next((cols[c] for c in ["source","src","u","from"]), None)
    dst_col = next((cols[c] for c in ["target","dst","v","to"]), None)
    dir_col = next((cols[c] for c in ["directed"]), None)
    w_eff   = next((cols[c] for c in ["effective_weight","weight","w"]), None)

    if not (src_col and dst_col):
        raise ValueError("No binary endpoint columns found; the view may be hyperedge-only. Try the generic edges_view export.")

    out = pl.DataFrame({
        "source": df[src_col],
        "target": df[dst_col],
        "weight": df[w_eff] if w_eff else pl.Series([1.0]*df.height),
        "directed": df[dir_col] if dir_col in df.columns else pl.Series([None]*df.height),
        "layer": pl.Series([layer]*df.height) if layer else pl.Series([None]*df.height),
    })
    out #.write_csv(path)

# Usage:
csv2_edge_list_path = "csv2_edge_list.csv"
export_edge_list_csv(G, csv2_edge_list_path, layer=None)
csv2_edge_list_path


'csv2_edge_list.csv'

In [47]:
# In-memory look at last few events
hist = G.history(as_df=True)   # DF [DataFrame]
hist.tail()

# Save to Parquet/CSV/JSON [JavaScript Object Notation]/NDJSON [Newline-Delimited JSON]
G.export_history("graph_history.parquet")


1278

In [49]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))

from graphglue.io import excel as graph_excel
from graphglue.core.graph import Graph


In [51]:
G = graph_excel.load_excel_to_graph("graph_input.xlsx", schema="auto")

In [52]:
G.global_entity_count()

60

In [59]:
from graphglue.adapters.networkx import to_backend, to_nx, from_nx

In [61]:
nxG, man = to_nx(G, directed=True, hyperedge_mode="skip")

TypeError: Graph.get_attr_vertex() missing 1 required positional argument: 'key'